In [4]:
from trueskill import Rating, quality_1vs1, rate_1vs1
import pandas as pd

In [6]:
df = pd.read_csv('inputs/ready_data/preprocessed_all_matches.csv')
df

,Division,Date,HomeTeam,AwayTeam,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,...,Difference_Overall_Score,Difference_Attack_Score,Difference_Middle_Score,Difference_Defensive_Score,Difference_Budget,DIFFERENCE_ELO,HISTORICAL_HOME_WINS,HISTORICAL_AWAY_WINS,HISTORICAL_DRAWS,HISTORICAL_MATCHES
0,SP1,2006-08-26,Valencia,Betis,1,1.0,0.0,H,14.0,8.0,...,5,7,6,5,20.0,21.596100,0,0,0,1
1,SP1,2006-08-27,Ath Bilbao,Sociedad,0,1.0,0.0,H,7.0,10.0,...,4,3,2,3,-1.5,-8.403900,0,0,0,1
2,SP1,2006-08-27,Espanol,Gimnastic,2,0.0,0.0,D,20.0,18.0,...,4,3,4,4,13.5,-38.403900,0,0,0,1
3,SP1,2006-08-27,La Coruna,Zaragoza,1,2.0,1.0,H,14.0,13.0,...,-1,-6,-2,3,3.0,21.596100,0,0,0,1
4,SP1,2006-08-27,Osasuna,Getafe,2,0.0,0.0,D,15.0,7.0,...,4,6,2,-1,6.5,-57.605850,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,SP1,2020-08-03,Valladolid,Ath Bilbao,2,0.0,2.0,A,19.0,10.0,...,-3,-2,-4,-3,-6.0,-61.242361,2,7,4,14
5206,SP1,2020-08-03,Levante,Granada,0,1.0,0.0,H,11.0,10.0,...,-3,-3,-3,-6,-15.4,28.004895,6,4,1,12
5207,SP1,2020-08-03,Villarreal,Leganes,2,1.0,0.0,H,15.0,12.0,...,2,1,2,1,2.7,135.588583,5,1,1,8
5208,SP1,2020-08-03,Betis,Real Madrid,1,1.0,1.0,D,14.0,13.0,...,4,5,4,3,28.7,-251.281817,4,14,3,22


In [7]:
Rating()

trueskill.Rating(mu=25.000, sigma=8.333)

In [11]:
r1 = Rating()
r2 = Rating()
print('{:.1%} chance to draw'.format(quality_1vs1(r2, r1)))


44.7% chance to draw


In [14]:
# P1 beats P2
new_r1, new_r2 = rate_1vs1(r1, r2)

# Draw
new_r1, new_r2 = rate_1vs1(r1, r2, drawn=True)

In [15]:
quality_1vs1(new_r1, new_r2)

0.37934094380495725

In [40]:
# Diccionario
trueskill_dict = {}
trueskill_season_dict = {}

home_elos = []
away_elos = []

home_mu = []
home_sigma = []

away_mu = []
away_sigma = []

home_quality_rate = []
away_quality_rate = []

draw_chance = []

home_elos_reset = []
away_elos_reset = []

home_mu_reset = []
home_sigma_reset = []

away_mu_reset = []
away_sigma_reset = []

home_quality_rate_reset = []
away_quality_rate_reset = []

draw_chance_reset = []

for ind, row in df.iterrows():
    home_team = row.HomeTeam
    away_team = row.AwayTeam
    result = row.Full_Time_Result
    season = row.Season

    if season not in trueskill_season_dict:
        trueskill_season_dict[season] = {}

    if home_team not in trueskill_dict:
        trueskill_dict[home_team] = Rating()
    
    if away_team not in trueskill_dict:
        trueskill_dict[away_team] = Rating()
    
    if home_team not in trueskill_season_dict[season]:
        trueskill_season_dict[season][home_team] = Rating()
    
    if away_team not in trueskill_season_dict[season]:
        trueskill_season_dict[season][away_team] = Rating()
    

    home_elos.append(trueskill_dict[home_team])
    away_elos.append(trueskill_dict[away_team])

    home_mu.append(trueskill_dict[home_team].mu)
    home_sigma.append(trueskill_dict[home_team].sigma)

    away_mu.append(trueskill_dict[away_team].mu)
    away_sigma.append(trueskill_dict[away_team].sigma)
    
    draw_chance.append(quality_1vs1(trueskill_dict[home_team], trueskill_dict[away_team]))


    home_elos_reset.append(trueskill_season_dict[season][home_team])
    away_elos_reset.append(trueskill_season_dict[season][away_team])

    home_mu_reset.append(trueskill_season_dict[season][home_team].mu)
    home_sigma_reset.append(trueskill_season_dict[season][home_team].sigma)

    away_mu_reset.append(trueskill_season_dict[season][away_team].mu)
    away_sigma_reset.append(trueskill_season_dict[season][away_team].sigma)
    
    draw_chance_reset.append(quality_1vs1(trueskill_season_dict[season][home_team], trueskill_season_dict[season][away_team]))



    if result == 0:
        trueskill_dict[home_team], trueskill_dict[away_team] = rate_1vs1(trueskill_dict[home_team], trueskill_dict[away_team], drawn=True)
        trueskill_season_dict[season][home_team], trueskill_season_dict[season][away_team] = rate_1vs1(trueskill_season_dict[season][home_team], trueskill_season_dict[season][away_team], drawn=True)
    elif result == 1:
        trueskill_dict[home_team], trueskill_dict[away_team] = rate_1vs1(trueskill_dict[home_team], trueskill_dict[away_team])
        trueskill_season_dict[season][home_team], trueskill_season_dict[season][away_team] = rate_1vs1(trueskill_season_dict[season][home_team], trueskill_season_dict[season][away_team])
    elif result == 2:
        trueskill_dict[away_team], trueskill_dict[home_team] = rate_1vs1(trueskill_dict[away_team], trueskill_dict[home_team])
        trueskill_season_dict[season][away_team], trueskill_season_dict[season][home_team] = rate_1vs1(trueskill_season_dict[season][away_team], trueskill_season_dict[season][home_team])
    else:
        raise Exception('This match has no result, please check the data.')

# df['HOME_TRUESKILL_NO_RESET'] = home_elos
# df['AWAY_TRUESKILL_NO_RESET'] = away_elos
df['HOME_TRUESKILL_MU_NO_RESET'] = home_mu
df['AWAY_TRUESKILL_MU_NO_RESET'] = away_mu
df['HOME_TRUESKILL_SIGMA_NO_RESET'] = home_sigma
df['AWAY_TRUESKILL_SIGMA_NO_RESET'] = away_sigma
df['DRAW_CHANCE_NO_RESET'] = draw_chance

# df['HOME_TRUESKILL_SEASON'] = home_elos_reset
# df['AWAY_TRUESKILL_SEASON'] = away_elos_reset
df['HOME_TRUESKILL_MU_SEASON'] = home_mu_reset
df['AWAY_TRUESKILL_MU_SEASON'] = away_mu_reset
df['HOME_TRUESKILL_SIGMA_SEASON'] = home_sigma_reset
df['AWAY_TRUESKILL_SIGMA_SEASON'] = away_sigma_reset
df['DRAW_CHANCE_SEASON'] = draw_chance_reset

In [46]:
df[['HomeTeam', 'AwayTeam', 
'Full_Time_Result', 
'HOME_TRUESKILL_NO_RESET', 'HOME_TRUESKILL_SEASON',
'AWAY_TRUESKILL_NO_RESET', 'AWAY_TRUESKILL_SEASON',
'DRAW_CHANCE_NO_RESET','DRAW_CHANCE_SEASON']].query('HomeTeam=="Betis" or AwayTeam=="Betis"')

,HomeTeam,AwayTeam,Full_Time_Result,HOME_TRUESKILL_NO_RESET,HOME_TRUESKILL_SEASON,AWAY_TRUESKILL_NO_RESET,AWAY_TRUESKILL_SEASON,DRAW_CHANCE_NO_RESET,DRAW_CHANCE_SEASON
0,Valencia,Betis,1,"trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.000, sigma=8.333)","trueskill.Rating(mu=25.000, sigma=8.333)",0.447214,0.447214
12,Betis,Ath Bilbao,1,"trueskill.Rating(mu=20.604, sigma=7.171)","trueskill.Rating(mu=20.604, sigma=7.171)","trueskill.Rating(mu=25.000, sigma=6.458)","trueskill.Rating(mu=25.000, sigma=6.458)",0.483204,0.483204
27,Sevilla,Betis,1,"trueskill.Rating(mu=32.144, sigma=6.312)","trueskill.Rating(mu=32.144, sigma=6.312)","trueskill.Rating(mu=25.643, sigma=6.038)","trueskill.Rating(mu=25.643, sigma=6.038)",0.462321,0.462321
30,Betis,Real Madrid,2,"trueskill.Rating(mu=23.963, sigma=5.519)","trueskill.Rating(mu=23.963, sigma=5.519)","trueskill.Rating(mu=29.178, sigma=5.281)","trueskill.Rating(mu=29.178, sigma=5.281)",0.527762,0.527762
45,Recreativo,Betis,1,"trueskill.Rating(mu=25.078, sigma=4.502)","trueskill.Rating(mu=25.078, sigma=4.502)","trueskill.Rating(mu=22.296, sigma=5.025)","trueskill.Rating(mu=22.296, sigma=5.025)",0.626867,0.626867
...,...,...,...,...,...,...,...,...,...
5169,Betis,Barcelona,2,"trueskill.Rating(mu=25.127, sigma=0.760)","trueskill.Rating(mu=24.874, sigma=1.668)","trueskill.Rating(mu=30.281, sigma=0.801)","trueskill.Rating(mu=28.963, sigma=1.826)",0.679226,0.751370
5176,Leganes,Betis,0,"trueskill.Rating(mu=24.126, sigma=0.794)","trueskill.Rating(mu=22.546, sigma=1.743)","trueskill.Rating(mu=25.088, sigma=0.762)","trueskill.Rating(mu=24.657, sigma=1.640)",0.970418,0.876846
5179,Betis,Mallorca,0,"trueskill.Rating(mu=25.072, sigma=0.760)","trueskill.Rating(mu=24.517, sigma=1.587)","trueskill.Rating(mu=24.080, sigma=0.769)","trueskill.Rating(mu=21.640, sigma=1.680)",0.970161,0.839565
5191,Valencia,Betis,1,"trueskill.Rating(mu=26.826, sigma=0.757)","trueskill.Rating(mu=25.987, sigma=1.542)","trueskill.Rating(mu=25.056, sigma=0.759)","trueskill.Rating(mu=24.336, sigma=1.539)",0.941837,0.906143
